In [17]:
import requests
import bs4 as BeautifulSoup
import pandas as pd
import os
from io import StringIO
pd.set_option('display.max_columns', None)

In [2]:
os.makedirs("team_stats", exist_ok=True)

Webiste used to scrape data = __Basketball Refrence__

In [3]:
years = list(range(1980,2025))
url_ = "https://www.basketball-reference.com/leagues/NBA_{}.html"


In [ ]:
for year in years:
    try:
        url = url_.format(year)
        response = requests.get(url)
        response.raise_for_status()  

        with open(f"team_stats/{year}.html", "w", encoding="utf-8") as f:
            f.write(response.text)
        print(f"Successfully saved data for {year}")

    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve data for {year}: {e}")

In [19]:
total_dfs = []
avg_dfs = []
advanced_dfs = []

for year in years:
        # Read the HTML file
        with open(f"team_stats/{year}.html", "r", encoding="utf-8") as f:
            page = f.read()
        soup = BeautifulSoup.BeautifulSoup(page, "html.parser")

        soup.find('div', class_="table_container").decompose()
        soup.find('tr', class_="over_header").decompose()
        


        total_stats_table = soup.find(id = "div_totals-team")  # contains the total stats of every team
        avg_stats_table = soup.find(id = "div_per_game-team") # data per game
        advanced_table = soup.find(id = "advanced-team") # advanced stats 
        if total_stats_table is None:
            print(f"Total stats table not found for year {year}")
        if avg_stats_table is None:
            print(f"Average stats table not found for year {year}")
        if advanced_table is None:
            print(f"Advanced stats table not found for year {year}")

        if total_stats_table and avg_stats_table and advanced_table:
                total_stats = pd.read_html(StringIO(str(total_stats_table)))[0]
                avg_stats = pd.read_html(StringIO(str(avg_stats_table)))[0]
                advanced_stats = pd.read_html(StringIO(str(advanced_table)))[0]
                
                # Add a Year column
                total_stats["Year"] = year
                avg_stats["Year"] = year
                advanced_stats["Year"] = year
                
                total_dfs.append(total_stats)
                avg_dfs.append(avg_stats)
                advanced_dfs.append(advanced_stats)
        else:
            print(f"Failed to process data for year {year}")

print(f"Total DFS: {len(total_dfs)}")
print(f"Avg DFS: {len(avg_dfs)}")
print(f"Advanced DFS: {len(advanced_dfs)}")



Total DFS: 45
Avg DFS: 45
Advanced DFS: 45


In [9]:
url_1= 'https://www.basketball-reference.com/playoffs'
data = requests.get(url_1)
try:
    with open ('team_stats/champions.html','w+',encoding="utf-8") as f:
        f.write(data.text)
    print("Succesfully saved data for champions")
except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve data for champions : {e}")

Succesfully saved data for champions


In [25]:
with open("team_stats/champions.html",encoding="utf-8") as f:
        page = f.read()
soup = BeautifulSoup.BeautifulSoup(page,'html.parser')
# decompose unneccessary parts of table
soup.find('tr', class_= "over_header").decompose()
champions_table = soup.find(id = "div_champions_index")
champions_stat = pd.read_html(StringIO(str(champions_table)))[0]

__Concatinating data and storing them in a df/csv__

In [21]:
total_stats_df = pd.concat(total_dfs)
total_stats_df.head()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1.0,San Antonio Spurs*,82,19755,3856,7738,0.498,52,206,0.252,3804,7532,0.505,2024,2528,0.801,1153,2515,3668,2326,771,333,1589,2103,9788,1980
1,2.0,Los Angeles Lakers*,82,19880,3898,7368,0.529,20,100,0.200,3878,7268,0.534,1622,2092,0.775,1085,2653,3738,2413,774,546,1639,1784,9438,1980
2,3.0,Cleveland Cavaliers,82,19930,3811,8041,0.474,36,187,0.193,3775,7854,0.481,1702,2205,0.772,1307,2381,3688,2108,764,342,1370,1934,9360,1980
3,4.0,New York Knicks,82,19780,3802,7672,0.496,42,191,0.220,3760,7481,0.503,1698,2274,0.747,1236,2303,3539,2265,881,457,1613,2168,9344,1980
4,5.0,Boston Celtics*,82,19880,3617,7387,0.490,162,422,0.384,3455,6965,0.496,1907,2449,0.779,1227,2457,3684,2198,809,308,1539,1974,9303,1980


In [26]:
avg_stats_df = pd.concat(avg_dfs)
advanced_stats_df = pd.concat(advanced_dfs)
champions_df = pd.DataFrame(champions_stat)


In [ ]:
os.makedirs("data",exist_ok=True)
total_stats_df.to_csv('data/total_stats.csv')
avg_stats_df.to_csv('data/avg_stats.csv')
advanced_stats_df.to_csv('data/advanced_stats.csv')
champions_df.to_csv('data/champions.csv')